<a href="https://colab.research.google.com/github/lmcanavals/pcd/blob/master/0602_readers_writers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt install spin

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'sudo apt autoremove' to remove them.
The following NEW packages will be installed:
  spin
0 upgraded, 1 newly installed, 0 to remove and 42 not upgraded.
Need to get 559 kB of archives.
After this operation, 1,776 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 spin amd64 6.4.6+dfsg-2 [559 kB]
Fetched 559 kB in 1s (712 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to front

In [10]:
%%file rw.pml
#define wait(s) atomic { s > 0 -> s-- }
#define signal(s) s++

byte mutex1 = 1
byte mutex2 = 1

byte numReaders = 0
byte numWriters = 0

active[2] proctype Writer() {
    do
    ::
      wait(mutex1)
      // write
      numWriters++
      assert numWriters <= 1
      assert numReaders == 0
      numWriters--
      signal(mutex1)
    od
}

active[5] proctype Reader() {
    do
    ::
      wait(mutex2)
      if
      :: numReaders == 0 -> wait(mutex1)
      :: else -> skip
      fi
      numReaders++
      signal(mutex2)

      // reading
      assert numWriters == 0

      wait(mutex2)
      numReaders--
      if
      :: numReaders == 0 -> signal(mutex1)
      :: else -> skip
      fi
      signal(mutex2)

    od
}

Overwriting rw.pml


In [11]:
%%bash
spin -a rw.pml && gcc pan.c && ./a.out
[ -f rw.pml.trail ] && spin -t rw.pml && rm *.trail


(Spin Version 6.4.6 -- 2 December 2016)
	+ Partial Order Reduction

Full statespace search for:
	never claim         	- (none specified)
	assertion violations	+
	acceptance   cycles 	- (not selected)
	invalid end states	+

State-vector 68 byte, depth reached 1221, errors: 0
     3643 states, stored
     4398 states, matched
     8041 transitions (= stored+matched)
        0 atomic steps
hash conflicts:        55 (resolved)

Stats on memory usage (in Megabytes):
    0.334	equivalent memory usage for states (stored*(State-vector + overhead))
    0.386	actual memory usage for states
  128.000	memory used for hash table (-w24)
    0.534	memory used for DFS stack (-m10000)
  128.827	total actual memory usage


unreached in proctype Writer
	rw.pml:21, state 12, "-end-"
	(1 of 12 states)
unreached in proctype Reader
	rw.pml:46, state 29, "-end-"
	(1 of 29 states)

pan: elapsed time 0 seconds
